In [1]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
os.chdir('../')
from model import *
os.chdir('../../data/')

In [2]:
transactions = pd.read_pickle('../data/compressed_data/transactions_train.pkl')
customers = pd.read_pickle('../data/compressed_data/customers.pkl')
articles = pd.read_pickle('../data/compressed_data/articles.pkl')

transactions['week'] = 104 - (transactions.t_dat.max() - transactions.t_dat).dt.days // 7

print('First week num: ', transactions.week.min(), '\nLast week num: ', transactions.week.max(), '\n')

First week num:  0 
Last week num:  104 



In [3]:
# Test week is week after last week in train data
test_week = transactions.week.max() + 1

# Filter transactions to last 10 weeks (most recent data)
transactions = transactions[transactions.week > transactions.week.max() - 10]

In [4]:
transactions.columns

Index(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id',
       'week'],
      dtype='object')

In [5]:
# Load radek's candidates
candidates_last_purchase = pd.read_csv('../data/candidates/radek_last_purchase.csv')
candidates_bestsellers = pd.read_csv('../data/candidates/radek_bestsellers.csv')
bestsellers_previous_week = pd.read_csv('../data/candidates/radek_bestsellers_previous_week.csv')

# Load my candidates

In [6]:
# Load my candidates
candidates_niw_loaded = pd.read_csv('../data/candidates_200_ranks/niw_candidates_colour_group_name.csv')

## Submission Loop -- check best k for kaggle

In [7]:
columns_to_use = [
    'article_id', 
    'product_type_no', 
    'graphical_appearance_no', 
    'colour_group_code', 
    'perceived_colour_value_id',
    'perceived_colour_master_id', 
    'department_no', 
    'index_code',
    'index_group_no', 
    'section_no', 
    'garment_group_no', 
    'FN', 
    'Active',
    'club_member_status', 
    'fashion_news_frequency', 
    'age', 
    'postal_code', 
    'bestseller_rank',
    'not_interacted_weekly_rank'
]

model_params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'boosting_type': 'dart',
    'n_estimators': 1,
    'importance_type': 'gain'
}

In [8]:
# Get bestsellers from previous week
bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week['week'] == bestsellers_previous_week['week'].max()]['article_id'].tolist()

niw_ranks = candidates_niw_loaded[['week', 'article_id', 'not_interacted_weekly_rank']].drop_duplicates()

for k in range(20, 201, 10):
    print('k = ', k)

    # Get top k similar not bought articles for each customer
    candidates_niw = candidates_niw_loaded.groupby(['week', 'customer_id']).head(k)\
        .drop(columns=['strategy', 'not_interacted_weekly_rank'])

    candidates_niw['week'] = candidates_niw['week'] + 1

    candidates_niw['t_dat'] = '2020-07-15'
    candidates_niw['price'] = 0
    candidates_niw['sales_channel_id'] = 2

    # Prepare data for model
    train_X, train_y, test_X, test, train_baskets = prepare_data(
        transactions,
        bestsellers_previous_week,
        candidates=[candidates_last_purchase, candidates_bestsellers, candidates_niw], 
        features=[customers, articles, niw_ranks], 
        cols_to_use=columns_to_use
        )
    
    del candidates_niw
    gc.collect()
    
    # Train model
    ranker = train_model(
        train_X, 
        train_y, 
        train_baskets, 
        model_params, 
        columns_to_use, 
        show_importance=10
    )

    gc.collect()

    # Make submission
    make_submission(customers, test, test_X, ranker, bestsellers_last_week, f'submission_niw_colour_group_{k}')

    del train_X, train_y, test_X, test, train_baskets, ranker
    gc.collect()



k =  20
Percentage of real transactions:  0.07205122701166096
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.264246 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1131
[LightGBM] [Info] Number of data points in the train set: 24896704, number of used features: 19
Feature importance:
bestseller_rank 0.8077098548474034
not_interacted_weekly_rank 0.10680044788940318
colour_group_code 0.0314940609368392
perceived_colour_master_id 0.01353900117930774
perceived_colour_value_id 0.012534448875430256
article_id 0.008222430606826677
garment_group_no 0.005870691149032165
index_code 0.005570038482793826
department_no 0.004991986469053238
product_type_no 0.003267039563910241
Starting submission process...
Calculating predictions...
Done.
Creating su

100%|██████████| 58.1M/58.1M [00:27<00:00, 2.24MB/s]


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  30
Percentage of real transactions:  0.058259282367467434
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.351032 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1116
[LightGBM] [Info] Number of data points in the train set: 31617411, number of used features: 19
Feature importance:
bestseller_rank 0.7790799892333308
not_interacted_weekly_rank 0.14366626665537352
colour_group_code 0.039277701527001135
article_id 0.010447098018257757
index_code 0.006679182887210959
garment_group_no 0.004347388643620012
perceived_colour_master_id 0.0034136583915778717
section_no 0.0032801294264810125
perceived_colour_value_id 0.00306747586781864
in

100%|██████████| 58.1M/58.1M [00:33<00:00, 1.83MB/s]  


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  40
Percentage of real transactions:  0.048813265276143256
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.494597 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1118
[LightGBM] [Info] Number of data points in the train set: 38420358, number of used features: 19
Feature importance:
bestseller_rank 0.7526329141373416
not_interacted_weekly_rank 0.17018041285685248
colour_group_code 0.04406308839472477
index_code 0.007519483173732698
product_type_no 0.005138237992154772
section_no 0.005059210256434062
perceived_colour_master_id 0.004603253313775617
article_id 0.004518404499914067
department_no 0.003318149247861335
garment_group_no 

...

Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  190
Percentage of real transactions:  0.014202789912926549
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 3.216797 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1060
[LightGBM] [Info] Number of data points in the train set: 141032599, number of used features: 19
Feature importance:
bestseller_rank 0.6462482400317187
not_interacted_weekly_rank 0.23144190643953944
colour_group_code 0.08418448615687522
perceived_colour_value_id 0.023368885799082854
perceived_colour_master_id 0.007645953797654852
index_code 0.0020890197178054484
department_no 0.0019903189211225945
article_id 0.0019768533233885653
section_no 0.0010543358128123179
g

100%|██████████| 57.8M/57.8M [00:26<00:00, 2.26MB/s]


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  200
Percentage of real transactions:  0.013559513782687575
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 3.293673 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1052
[LightGBM] [Info] Number of data points in the train set: 147883693, number of used features: 19
Feature importance:
bestseller_rank 0.6448497293239519
not_interacted_weekly_rank 0.23211452813484848
colour_group_code 0.08603873207334307
perceived_colour_value_id 0.022603163479539466
perceived_colour_master_id 0.00818751815502367
index_code 0.002185755223015063
department_no 0.002130059828091432
article_id 0.0018905137821869122
garment_group_no 0.0
FN 0.0
Starting 

100%|██████████| 57.8M/57.8M [00:26<00:00, 2.28MB/s]


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.


In [9]:
!kaggle competitions submissions -c h-and-m-personalized-fashion-recommendations | head -n 21

fileName                                 date                 description                       status    publicScore  privateScore  
---------------------------------------  -------------------  --------------------------------  --------  -----------  ------------  
submission_niw_colour_group_200.csv.gz   2023-12-14 20:20:44  submission_niw_colour_group_200   pending                              
submission_niw_colour_group_190.csv.gz   2023-12-14 20:08:46  submission_niw_colour_group_190   complete  0.0156       0.01542       
submission_niw_colour_group_180.csv.gz   2023-12-14 19:57:38  submission_niw_colour_group_180   complete  0.01562      0.01543       
submission_niw_colour_group_170.csv.gz   2023-12-14 19:46:27  submission_niw_colour_group_170   complete  0.01537      0.0151        
submission_niw_colour_group_160.csv.gz   2023-12-14 19:35:46  submission_niw_colour_group_160   complete  0.01541      0.01513       
submission_niw_colour_group_150.csv.gz   2023-12-14 19:25:03  